#### Approaches : 
- first, open the sheet subnational 1 tree cover loss 
- drop the threshold and area_ha column since we tend to analyze the dataset by just province 
- Pivot(transpose) columns into feature by year 
- Do the same thing with carbon sheet
- merge the data

In [1]:
import pandas as pd 
import numpy as np 

import os, sys
if os.path.abspath('../') not in sys.path:
    sys.path.insert(0, os.path.abspath('../'))

Open the datasheet
#### We will only use the subnational following by feature values from 2015 to 2022

In [2]:
forest_data = pd.read_excel('../data/LoveWaterdata.xlsx', sheet_name='Subnational 1 tree cover loss')
forest_data = forest_data[['subnational1', 'tc_loss_ha_2015', 'tc_loss_ha_2016',
                            'tc_loss_ha_2017', 'tc_loss_ha_2018', 'tc_loss_ha_2019', 'tc_loss_ha_2020',
                            'tc_loss_ha_2021']]

In [3]:
forest_data 

,subnational1,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021
0,An Giang,70,264,253,130,45,16,55
1,An Giang,31,129,130,77,24,6,20
2,An Giang,30,120,121,74,23,6,19
3,An Giang,28,113,117,71,22,6,18
4,An Giang,27,106,109,68,21,5,17
...,...,...,...,...,...,...,...,...
499,Đồng Tháp,54,196,97,58,9,1,2
500,Đồng Tháp,51,170,79,46,6,1,2
501,Đồng Tháp,49,161,74,44,6,1,2
502,Đồng Tháp,32,76,36,25,4,1,1


## group the threshold by the sum of forest lost in each year  

In [4]:
# group by the total tree cover loss by subnational1
forest_data = forest_data.groupby(['subnational1']).sum().reset_index()
forest_data

,subnational1,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021
0,An Giang,229,904,930,540,174,48,155
1,Bà Rịa - Vũng Tàu,4254,6175,6594,7409,4889,5628,4156
2,Bình Dương,53875,58556,76933,47980,35862,30948,27677
3,Bình Phước,30828,68985,77781,43436,45142,35805,51861
4,Bình Thuận,8982,8885,8934,7088,8741,8506,6660
...,...,...,...,...,...,...,...,...
58,Đà Nẵng,3605,7473,7810,7599,7742,6294,3740
59,Đắk Lắk,36984,64091,66965,48950,38843,44177,33365
60,Đắk Nông,35265,52574,36306,21161,21632,15006,13736
61,Đồng Nai,16251,21422,25487,27123,34202,32281,29809


In [5]:
forest_data.columns 

Index(['subnational1', 'tc_loss_ha_2015', 'tc_loss_ha_2016', 'tc_loss_ha_2017',
       'tc_loss_ha_2018', 'tc_loss_ha_2019', 'tc_loss_ha_2020',
       'tc_loss_ha_2021'],
      dtype='object')

In [6]:
#describe transpose the data
forest_data.describe().T

,count,mean,std,min,25%,50%,75%,max
tc_loss_ha_2015,63.0,27954.984127,38424.435709,0.0,560.5,14593.0,43508.0,213781.0
tc_loss_ha_2016,63.0,45140.809524,54065.761804,0.0,1752.0,26582.0,69082.0,239219.0
tc_loss_ha_2017,63.0,37040.793651,41935.653826,2.0,1241.5,25487.0,67292.5,192872.0
tc_loss_ha_2018,63.0,29164.000000,33532.190388,0.0,619.0,16081.0,48856.0,149429.0
tc_loss_ha_2019,63.0,28247.031746,33331.944501,0.0,566.0,15945.0,45518.5,154043.0
tc_loss_ha_2020,63.0,26989.015873,31795.367461,0.0,697.0,15006.0,43985.0,137689.0
tc_loss_ha_2021,63.0,25577.666667,31756.081126,0.0,419.5,14934.0,39334.0,146415.0


In [7]:
forest_feature = pd.melt(forest_data.drop('subnational1', axis=1), var_name='tc_loss_year', value_name='tc_loss_ha')
forest_feature['tc_loss_year'] = forest_feature['tc_loss_year'].str[-4:]

In [8]:
provinces = forest_data['subnational1']
forest_feature = forest_data.drop('subnational1', axis=1)

In [9]:
forest_feature

,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021
0,229,904,930,540,174,48,155
1,4254,6175,6594,7409,4889,5628,4156
2,53875,58556,76933,47980,35862,30948,27677
3,30828,68985,77781,43436,45142,35805,51861
4,8982,8885,8934,7088,8741,8506,6660
...,...,...,...,...,...,...,...
58,3605,7473,7810,7599,7742,6294,3740
59,36984,64091,66965,48950,38843,44177,33365
60,35265,52574,36306,21161,21632,15006,13736
61,16251,21422,25487,27123,34202,32281,29809


In [10]:
forest_pivot = pd.melt(forest_feature, var_name='tc_loss_year', value_name='tc_loss_ha')
forest_pivot


,tc_loss_year,tc_loss_ha
0,tc_loss_ha_2015,229
1,tc_loss_ha_2015,4254
2,tc_loss_ha_2015,53875
3,tc_loss_ha_2015,30828
4,tc_loss_ha_2015,8982
...,...,...
436,tc_loss_ha_2021,3740
437,tc_loss_ha_2021,33365
438,tc_loss_ha_2021,13736
439,tc_loss_ha_2021,29809


### extract years from the tc_loss_XXXX --> XXXX

In [11]:
years = forest_pivot['tc_loss_year'].values
values = forest_pivot['tc_loss_ha'].values
for i in range(len(years)):
    year = years[i][-4:]  
    years[i] = year

##### --> To merge : years, values and provinces  

### next, open the carbon data sheet 

In [12]:
carbon_data = pd.read_excel('../data/LoveWaterdata.xlsx', sheet_name='Subnational 1 carbon data')
carbon_data = carbon_data[['subnational1', 
                           'gfw_forest_carbon_gross_emissions_2015__Mg_CO2e', 'gfw_forest_carbon_gross_emissions_2016__Mg_CO2e', 'gfw_forest_carbon_gross_emissions_2017__Mg_CO2e', 'gfw_forest_carbon_gross_emissions_2018__Mg_CO2e', 
                           'gfw_forest_carbon_gross_emissions_2019__Mg_CO2e', 'gfw_forest_carbon_gross_emissions_2020__Mg_CO2e', 'gfw_forest_carbon_gross_emissions_2021__Mg_CO2e']]

In [13]:
carbon_data = carbon_data.groupby(['subnational1']).sum().reset_index()
carbon_data

,subnational1,gfw_forest_carbon_gross_emissions_2015__Mg_CO2e,gfw_forest_carbon_gross_emissions_2016__Mg_CO2e,gfw_forest_carbon_gross_emissions_2017__Mg_CO2e,gfw_forest_carbon_gross_emissions_2018__Mg_CO2e,gfw_forest_carbon_gross_emissions_2019__Mg_CO2e,gfw_forest_carbon_gross_emissions_2020__Mg_CO2e,gfw_forest_carbon_gross_emissions_2021__Mg_CO2e
0,An Giang,24733.0,110738.0,200469.0,126319.0,34024.0,6892.0,38233.0
1,Bà Rịa - Vũng Tàu,665080.0,939096.0,1005701.0,1181104.0,417687.0,592491.0,868471.0
2,Bình Dương,9878470.0,10382636.0,13444575.0,8900967.0,6173317.0,5540256.0,7870561.0
3,Bình Phước,5528408.0,11982252.0,12815587.0,7068943.0,6765024.0,5538039.0,12505463.0
4,Bình Thuận,1240072.0,1366015.0,1800787.0,1057538.0,666382.0,809123.0,2365391.0
...,...,...,...,...,...,...,...,...
58,Đà Nẵng,673595.0,1610895.0,1758511.0,1703073.0,1714335.0,1383690.0,857068.0
59,Đắk Lắk,7381107.0,14259813.0,12967426.0,9992466.0,7106930.0,8758557.0,7437228.0
60,Đắk Nông,7827816.0,11550485.0,7978319.0,4361978.0,4675238.0,3142453.0,3205878.0
61,Đồng Nai,2804517.0,3926043.0,5841910.0,5195976.0,4440935.0,4752050.0,7157571.0


In [14]:
carbon_features = carbon_data.drop('subnational1', axis=1)
carbon_pivot = pd.melt(carbon_features, var_name='carbon_year', value_name='carbon_gross_emissions')
carbon_pivot

,carbon_year,carbon_gross_emissions
0,gfw_forest_carbon_gross_emissions_2015__Mg_CO2e,24733.0
1,gfw_forest_carbon_gross_emissions_2015__Mg_CO2e,665080.0
2,gfw_forest_carbon_gross_emissions_2015__Mg_CO2e,9878470.0
3,gfw_forest_carbon_gross_emissions_2015__Mg_CO2e,5528408.0
4,gfw_forest_carbon_gross_emissions_2015__Mg_CO2e,1240072.0
...,...,...
436,gfw_forest_carbon_gross_emissions_2021__Mg_CO2e,857068.0
437,gfw_forest_carbon_gross_emissions_2021__Mg_CO2e,7437228.0
438,gfw_forest_carbon_gross_emissions_2021__Mg_CO2e,3205878.0
439,gfw_forest_carbon_gross_emissions_2021__Mg_CO2e,7157571.0


In [15]:
emissions_years = carbon_pivot['carbon_year'].values
carbon_values = carbon_pivot['carbon_gross_emissions'].values

In [16]:
carbon_values = pd.DataFrame(carbon_values)
values = pd.DataFrame(values)
years =pd.DataFrame(years)
column_names = ['carbon_gross_emissions', 'tc_loss_ha', 'years']

# Rename columns for each DataFrame 0 by name 
carbon_values.rename(columns={0: column_names[0]}, inplace=True)
values.rename(columns={0: column_names[1]}, inplace=True)
years.rename(columns={0: column_names[2]}, inplace=True)

print(provinces.shape, carbon_values.shape, values.shape, years.shape)


(63,) (441, 1) (441, 1) (441, 1)


In [17]:
deforest_carbon = pd.concat([carbon_values, values, years], axis=1)
deforest_carbon

,carbon_gross_emissions,tc_loss_ha,years
0,24733.0,229,2015
1,665080.0,4254,2015
2,9878470.0,53875,2015
3,5528408.0,30828,2015
4,1240072.0,8982,2015
...,...,...,...
436,857068.0,3740,2021
437,7437228.0,33365,2021
438,3205878.0,13736,2021
439,7157571.0,29809,2021


In [18]:
# create a province column code hoi ngu 
deforest_carbon['Province'] = [
    'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city','Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai',
     'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city','Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai',
     'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city','Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai',
     'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city','Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai',
     'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city','Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai',
     'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city','Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai',
     'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Ho Chi Minh city','Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai'
]


In [19]:
deforest_carbon

,carbon_gross_emissions,tc_loss_ha,years,Province
0,24733.0,229,2015,An Giang
1,665080.0,4254,2015,Ba Ria-Vung Tau
2,9878470.0,53875,2015,Bac Lieu
3,5528408.0,30828,2015,Bac Kan
4,1240072.0,8982,2015,Bac Giang
...,...,...,...,...
436,857068.0,3740,2021,Tra Vinh
437,7437228.0,33365,2021,Tuyen Quang
438,3205878.0,13736,2021,Vinh Long
439,7157571.0,29809,2021,Vinh Phuc


In [20]:
# save into excel 
deforest_carbon.to_excel('../data/deforest_carbon.xlsx', index=False)